<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/AI_Analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Work in Porgress. A template for the AI Analyst pipeline.  The genral idea is to analyse set of documents (product reviews, emails, etc.,)**

In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('OpenAI')

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import copy
import json
from openai import OpenAI
#from openai import ChatCompletion

#from utils_local import o1_tools

client = OpenAI(api_key=openai_api_key)
#O1_MODEL = 'o1-mini'
GPT_MODEL = 'gpt-4o-mini'

In [ ]:
# System prompt for the execution model
gpt4o_system_prompt = """
You are a helpful data analyst responsible for analysis provided data file using the tools provided.
You must explain your decision-making process across various steps. if ask to read data, you should read the
data file using the tool provided. The data file is located at /content/Beer_small_test.csv
If you are not asked to read the data file, you should answer a user question to the best of your ability.

"""

In [ ]:
# Tools
def read_csv_to_dataframe(file_path: str):
    import pandas as pd
    try:
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        return f"Error reading CSV file: {e}"

In [ ]:
TOOLS_test = [
    {
        "type": "function",
        "function": {
            "name": "read_csv_to_dataframe",
            "description":"Reads a CSV file and returns the DataFrame",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_path": {"type": "string", "description": "Path to the CSV file."}
                },
                "required": ["file_path"],
                "additionalProperties": False,
            },
        },
    }]

In [ ]:
data = '/content/Beer_small_test.csv'

messages = [
    {"role": "system", "content": 'you are helful data analyst reading the data files'},
    {"role": "user", "content": f"please read the data file"}]

In [ ]:
content = '''You are an assistant that uses tools if asked anything that can be addressed by a tool.
              or directly answer user questions as asked
'''
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": gpt4o_system_prompt},
        {"role": "user", "content": f"please read the data file"}
    ],
    tools=TOOLS_test,
    tool_choice="auto"
)

In [ ]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_JvES3EhPvvp72lNLgybB6hYS', function=Function(arguments='{"file_path":"/content/Beer_small_test.csv"}', name='read_csv_to_dataframe'), type='function')]

In [ ]:
response.choices[0].message.tool_calls[0].function.name

'read_csv_to_dataframe'

In [ ]:
response.choices[0].message.tool_calls[0].function.arguments

'{"file_path":"/content/Beer_small_test.csv"}'

In [ ]:
if response.choices[0].message.tool_calls:
  # Assuming response.choices[0].message.tool_calls[0] is stored in a variable `tool_call`
  tool_call = response.choices[0].message.tool_calls[0]

  # Extract the function name and arguments
  function_name = tool_call.function.name  # Access the function name
  function_arguments = tool_call.function.arguments  # Access the function arguments

  # Parse the JSON string
  parsed_arguments = json.loads(function_arguments)

  print("Function Name:", function_name)
  print("Function Arguments:", function_arguments)

  file_path = parsed_arguments.get("file_path")

  print("File Path:", file_path)
  #function_callable = globals().get(function_name)
  #df = function_callable(function_arguments)
else:
  print(f'model response: {response.choices[0].message.content}')

Function Name: read_csv_to_dataframe
Function Arguments: {"file_path":"/content/Beer_small_test.csv"}
File Path: /content/Beer_small_test.csv


In [ ]:
function_callable = globals().get(function_name)
df = function_callable(file_path)
df.head()

,string_name,string_beer_id,string_brewer_id,number_abv,string_style,number_appearance,number_aroma,number_palate,number_taste,ratings,date_time,string_profile_name,text
0,Sausa Weizen,47986,10325,5.0,Hefeweizen,2.5,2.0,1.5,1.5,1.5,2009-02-16T15:57:03Z,stcules,A lot of foam. But a lot.\tIn the smell some b...
1,Red Moon,48213,10325,6.2,English Strong Ale,3.0,2.5,3.0,3.0,3.0,2009-03-01T08:44:57Z,stcules,"Dark red color, light beige foam, average.\tIn..."
2,Black Horse Black Beer,48215,10325,6.5,Foreign / Export Stout,3.0,2.5,3.0,3.0,3.0,2009-03-01T09:10:04Z,stcules,"Almost totally black. Beige foam, quite compac..."
3,Sausa Pils,47969,10325,5.0,German Pilsener,3.5,3.0,2.5,3.0,3.0,2009-02-15T14:12:25Z,stcules,"Golden yellow color. White, compact foam, quit..."
4,Cauldron DIPA,64883,1075,7.7,American Double / Imperial IPA,4.0,4.5,4.0,4.5,4.0,2010-12-30T13:53:26Z,johnmichaelsen,"According to the website, the style for the Ca..."


In [ ]:
response.choices[0].message.content

'The meaning of life is a philosophical question that has been pondered for centuries. Different cultures, religions, and individuals have various interpretations and beliefs regarding this topic. Some people find meaning through religion or spirituality, others through relationships and connections with family and friends, and some find purpose in pursuits of knowledge, creativity, or contribution to society. Ultimately, the meaning of life can be deeply personal and subjective, and what is meaningful to one person may not be the same for another.'

In [ ]:
#test block - 2nd model call after file read
# Convert the DataFrame to a string (e.g., CSV format)
df_truncated = df.head(10)
data_string = df_truncated.to_csv(index=False)

# Define the system and user prompts
system_prompt_tools = "You are a data analyst. Analyze the provided dataset and summarize key insights."
user_prompt_tools = f"The dataset is as follows:\n\n{data_string}\n\nPlease provide a summary of the data."

# Call the GPT-4o model
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt_tools},
        {"role": "user", "content": user_prompt_tools},
    ]
)

# Print the model's response
response.choices[0].message.content

In [ ]:
response.choices[0].message.content

'The dataset consists of beer reviews, capturing various attributes and ratings from different users. Here\'s a summary of the key insights from the data:\n\n1. **Beers Reviewed:**\n   - The dataset includes reviews for seven different beers, each associated with specific IDs for the beer and the brewer.\n\n2. **Beer Characteristics:**\n   - Beers span a variety of styles, including Hefeweizen, English Strong Ale, Foreign/Export Stout, German Pilsener, American Double/Imperial IPA, and Herbed/Spiced Beer.\n   - The ABV (alcohol by volume) ranges from 4.7% to 7.7%.\n\n3. **Rating Attributes:**\n   - Ratings are provided for appearance, aroma, palate, taste, and overall impressions for each beer.\n   - The ratings are numerical, generally on a scale from 1 to 5, with varying degrees of detailed scoring.\n\n4. **Review Timeframe:**\n   - Reviews span several years, with the earliest from 2009 and the most recent entries up to 2012.\n\n5. **Reviewers:**\n   - Reviews are provided by severa

In [ ]:
print(response.choices[0].message.content)

The dataset consists of beer reviews, capturing various attributes and ratings from different users. Here's a summary of the key insights from the data:

1. **Beers Reviewed:**
   - The dataset includes reviews for seven different beers, each associated with specific IDs for the beer and the brewer.

2. **Beer Characteristics:**
   - Beers span a variety of styles, including Hefeweizen, English Strong Ale, Foreign/Export Stout, German Pilsener, American Double/Imperial IPA, and Herbed/Spiced Beer.
   - The ABV (alcohol by volume) ranges from 4.7% to 7.7%.

3. **Rating Attributes:**
   - Ratings are provided for appearance, aroma, palate, taste, and overall impressions for each beer.
   - The ratings are numerical, generally on a scale from 1 to 5, with varying degrees of detailed scoring.

4. **Review Timeframe:**
   - Reviews span several years, with the earliest from 2009 and the most recent entries up to 2012.

5. **Reviewers:**
   - Reviews are provided by several users, such as “s

# Topics extraction

In [ ]:
%%capture
!pip install anthropic

In [ ]:
from google.colab import userdata
claude_api_key = userdata.get('Antropic')

In [ ]:
import pandas as pd
import random
from anthropic import Anthropic

In [ ]:
df_context = pd.read_csv("/content/Mejuri_Retail.csv",  usecols=['Text'])

In [ ]:
#test block
df_context.iloc[1000,0]

In [ ]:
#get the context
context = ''
count = 0
while count<150000:
  number = random.randint(1, 17850)
  context+=df_context.iloc[number,0]
  length=len(context.split())
  count+=length

In [ ]:
print("here is the context to be used:/n/n " + context)

In [ ]:
client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=Antropic,
)

message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=8192,
    temperature=1,
    messages=[{"role": "user", "content": "Hello, world"}]
)
print(message.content)

In [ ]:
system_prompt = '''
You are a data analyst providing data analysis for the executive team.
Based on the provided context,
please create a list of top level themes and subthemes for sentiment analysis,
summarization, and trend analysis. for each theme create a list of keywords or
phrases that make up that theme/sub themes.

refer to the following policy for theme creation.

- the theme has to be in the review text itself
- the theme has to be descriptive/actionable
- the theme should be understood by executives, corporate analysts,
and entry level personell of the product or service company mentioned
in the feedback
'''

In [ ]:
client = Anthropic(api_key=claude_api_key) #defaults to os.environ.get("ANTHROPIC_API_KEY"

message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=8192,
    temperature=0,
    system=system_prompt,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "here is the context to be used:/n/n " + context
                }
            ]
        }
    ]
)

In [ ]:
cleaned_text = ' '.join(message.content[0].text.split())
cleaned_text

# Orchestration Layer

In [ ]:
%%capture
!pip install anthropic

In [ ]:
from google.colab import userdata
api_key_anthropic = userdata.get('Antropic')
#api_key_openAI = userdata.get('OpenAI')
#api_key_gemini = userdata.get('google')

In [ ]:
import anthropic
from typing import Literal

class QueryOrchestrator:
    """
    Orchestrator that routes queries between RAG and Agent pipelines.
    Uses Claude's extended thinking to reason through routing decisions.
    """

    def __init__(self, api_key: str):
        """
        Initialize the orchestrator with Anthropic API key.

        Args:
            api_key: Anthropic API key for Claude access
        """
        self.client = anthropic.Anthropic(api_key=api_key)
        self.model = "claude-sonnet-4-20250514"  # Latest model with thinking support

    def route_query(self, user_query: str, context_info: str = "") -> Literal["RAG", "Agent"]:
        """
        Route a user query to either RAG or Agent pipeline.

        Args:
            user_query: The user's question or request
            context_info: Optional context about the document corpus

        Returns:
            "RAG" or "Agent" based on the routing decision
        """

        system_prompt = self._build_system_prompt()
        user_message = self._build_user_message(user_query, context_info)

        try:
            response = self.client.messages.create(
                model=self.model,
                max_tokens=16000,
                thinking={
                    "type": "enabled",
                    "budget_tokens": 10000
                },
                messages=[{
                    "role": "user",
                    "content": user_message
                }],
                system=system_prompt
            )

            # Extract the final decision from response
            decision = self._extract_decision(response)

            # Optional: Extract and log thinking process
            thinking_content = self._extract_thinking(response)
            if thinking_content:
                print(f"\n[THINKING PROCESS]:\n{thinking_content[:500]}...\n")

            return decision

        except Exception as e:
            print(f"Error during routing: {e}")
            # Default to RAG on error as it's safer for general queries
            return "RAG"

    def _build_system_prompt(self) -> str:
        """Build the system prompt with routing instructions."""
        return """You are an intelligent query routing system for a data processing platform.

Your task is to analyze user queries and route them to the appropriate processing pipeline:

**RAG Pipeline**: Use when the query can be answered by:
- Directly retrieving and synthesizing information from document text
- Finding specific facts, quotes, or passages in the documents
- Summarizing content from the documents
- Answering questions about what the documents say or contain
- General information retrieval tasks

**Agent Pipeline**: Use when the query requires analytical tools. The Agent has these tools:
1. Analyze Text Metadata - examines document metadata (dates, authors, sources, etc.)
2. Extract Keywords from Text Corpus - identifies key terms across documents
3. Return Keyword Sentiments - analyzes sentiment associated with specific keywords
4. Count Keyword Mentions - counts occurrences of keywords in the corpus

Route to Agent when the query needs:
- Metadata analysis (dates, document counts, author information)
- Keyword extraction or keyword-based analysis
- Sentiment analysis of specific terms or topics
- Frequency counts or statistical analysis of term mentions
- Trends or patterns across documents requiring computation

**Critical Guidelines**:
- If a query asks "what do the documents say about X", use RAG (content retrieval)
- If a query asks "how often is X mentioned" or "what's the sentiment around X", use Agent (analysis tools)
- Default to RAG for straightforward information retrieval
- Choose Agent only when tools are clearly needed

Think through your decision carefully, then respond with ONLY one word: either "RAG" or "Agent"."""

    def _build_user_message(self, user_query: str, context_info: str) -> str:
        """Build the user message with query and context."""
        message = f"User Query: {user_query}"

        if context_info:
            message += f"\n\nDocument Corpus Context: {context_info}"

        message += "\n\nAnalyze this query and determine whether to route it to RAG or Agent. Respond with only 'RAG' or 'Agent'."

        return message

    def _extract_decision(self, response) -> Literal["RAG", "Agent"]:
        """Extract the routing decision from Claude's response."""
        for block in response.content:
            if block.type == "text":
                text = block.text.strip().upper()
                if "RAG" in text and "AGENT" not in text:
                    return "RAG"
                elif "AGENT" in text:
                    return "Agent"

        # Default to RAG if unclear
        print("Warning: Could not clearly determine routing, defaulting to RAG")
        return "RAG"

    def _extract_thinking(self, response) -> str:
        """Extract thinking content for debugging/logging."""
        for block in response.content:
            if block.type == "thinking":
                return block.thinking
        return ""


# Example usage
if __name__ == "__main__":
    # Initialize orchestrator
    api_key = "your-api-key-here"
    orchestrator = QueryOrchestrator(api_key)

    # Example queries
    test_queries = [
        "What are the main complaints mentioned in the customer reviews?",
        "How many times is 'pricing' mentioned across all emails?",
        "Summarize the feedback from the product reviews",
        "What's the sentiment around 'customer service' in the documents?",
        "Extract the top 10 keywords from the email corpus",
        "What did John say about the new feature in his email?",
        "Analyze the metadata to find how many documents were sent in Q1",
        "Which documents discuss the pricing strategy?"
    ]

    context = "Collection of customer emails and product reviews from 2024"

    print("Query Routing Results:")
    print("=" * 60)

    for query in test_queries:
        decision = orchestrator.route_query(query, context)
        print(f"\nQuery: {query}")
        print(f"Route: {decision}")
        print("-" * 60)